In [1]:
import pandas as pd
import os

path = "/home/secludor/langchain-kr/RAG-Practices/GAS5 CP/"
file = "공격성&반복_0_0724_masked"

pd.set_option("display.max_column", 30)
data = pd.read_csv(os.path.join(path, file) + ".csv")
data.head(1)

,Unnamed: 0,number,title,name,date,answer_state,contents,text_morphed_masked,text_masked,has_attachment,answer_contents,charge,answer_date,registration_number,charge_name,title_and_contents,aggr,욕설_모욕,비꼼_시비,성희롱,요지불명,저격성 민원,완전중복,반복,출처
0,0,85180,동구문화센터 건너편에 신호등을 보강해주시면 감사하겠습니다.,신○○,2021-04-14,이송이첩,안녕하세요 관리자님. 동구문화센터 인근 학교에 재직 중인 교사입니다. 동구문화센터 ...,안녕 하 관리자 `지역` 문화센터 인근 학교 재직 교사 `지역` 문화센터 근방 신호...,안녕 하시 어요 관리자 님. `지역` 문화센터 인근 학교 에 재직 중 이ㄴ 교사 ...,False,NaN,※ 연계처리내역 : 민원상세보기(원본)\n * 추천기관 : 경찰청\n * 이송(추천...,NaN,NaN,NaN,동구문화센터 건너편에 신호등을 보강해주시면 감사하겠습니다. 안녕하세요 관리자님. 동...,0,0,0,0,0,0,False,0,광주광역시_동구


In [2]:
data.describe()

,Unnamed: 0,number,registration_number,aggr,욕설_모욕,비꼼_시비,성희롱,요지불명,저격성 민원,반복
count,3256.000000,3256.000000,3.010000e+03,3256.0,3256.0,3256.0,3256.0,3256.0,3256.000000,3256.0
mean,1627.720516,122498.632064,2.016980e+17,0.0,0.0,0.0,0.0,0.0,0.001536,0.0
std,940.372720,183365.999654,5.066812e+14,0.0,0.0,0.0,0.0,0.0,0.039163,0.0
min,0.000000,1.000000,2.000000e+17,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
25%,813.750000,3200.000000,2.010000e+17,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
50%,1627.500000,33497.500000,2.020000e+17,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
75%,2441.250000,117874.000000,2.020000e+17,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
max,3257.000000,567975.000000,2.020000e+17,0.0,0.0,0.0,0.0,0.0,1.000000,0.0


제목과 내용 병합

In [2]:
data["text"] = "#" + data["title"] + "\n" + data["contents"]

임베딩할 컬럼 확인

In [3]:
to_embed = ["text", "text_masked", "text_morphed_masked"]
# data['text']
# data['text_masked']
# data['text_morphed_masked']

임베딩을 위해 길이 확인

In [4]:
for i in range(len(data)):
    if not isinstance(data.text.iloc[i], str):
        print(i)

In [5]:
for col in to_embed:
    data[f"len_{col}"] = data[col].apply(func=len)

In [6]:
data.shape

(3256, 29)

In [20]:
data_able = data
for col in to_embed:
    print(col)
    print(data_able[data_able[f"len_{col}"] > 6500].number)

text
180      2712
1181    82681
3065     1244
Name: number, dtype: int64
text_masked
180      2712
210      2032
1181    82681
2028    33773
3065     1244
Name: number, dtype: int64
text_morphed_masked
1181    82681
Name: number, dtype: int64


In [21]:
data_able = data
for col in to_embed:
    data_able = data_able[data_able[f"len_{col}"] < 6500]

In [22]:
data_able.describe()

,Unnamed: 0,number,registration_number,aggr,욕설_모욕,비꼼_시비,성희롱,요지불명,저격성 민원,반복,len_text,len_text_masked,len_text_morphed_masked
count,3251.000000,3251.000000,3.006000e+03,3251.0,3251.0,3251.0,3251.0,3251.0,3251.000000,3251.0,3251.000000,3251.000000,3251.000000
mean,1628.173793,122649.370655,2.016986e+17,0.0,0.0,0.0,0.0,0.0,0.001538,0.0,456.330975,516.101815,292.358044
std,940.026898,183462.477497,5.065081e+14,0.0,0.0,0.0,0.0,0.0,0.039193,0.0,459.812158,543.958467,301.916048
min,0.000000,1.000000,2.000000e+17,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,16.000000,7.000000,1.000000
25%,814.500000,3218.500000,2.010000e+17,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,183.000000,196.000000,113.500000
50%,1628.000000,33510.000000,2.020000e+17,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,327.000000,360.000000,206.000000
75%,2441.500000,118035.000000,2.020000e+17,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,558.000000,629.500000,359.000000
max,3257.000000,567975.000000,2.020000e+17,0.0,0.0,0.0,0.0,0.0,1.000000,0.0,4891.000000,5705.000000,3295.000000


임베딩

In [23]:
from langchain_upstage import UpstageEmbeddings

p_embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

# from langchain_openai import OpenAIEmbeddings
# openai_embeddings = OpenAIEmbeddings(
#     model="text-embedding-3-large"
# )

In [24]:
for col in to_embed:
    text_embedded = p_embeddings.embed_documents(texts=list(data_able[col]))
    data_able[f"{col}_embedded"] = text_embedded
data_able.head(2)

,Unnamed: 0,number,title,name,date,answer_state,contents,text_morphed_masked,text_masked,has_attachment,answer_contents,charge,answer_date,registration_number,charge_name,...,욕설_모욕,비꼼_시비,성희롱,요지불명,저격성 민원,완전중복,반복,출처,text,len_text,len_text_masked,len_text_morphed_masked,text_embedded,text_masked_embedded,text_morphed_masked_embedded
0,0,85180,동구문화센터 건너편에 신호등을 보강해주시면 감사하겠습니다.,신○○,2021-04-14,이송이첩,안녕하세요 관리자님. 동구문화센터 인근 학교에 재직 중인 교사입니다. 동구문화센터 ...,안녕 하 관리자 `지역` 문화센터 인근 학교 재직 교사 `지역` 문화센터 근방 신호...,안녕 하시 어요 관리자 님. `지역` 문화센터 인근 학교 에 재직 중 이ㄴ 교사 ...,False,NaN,※ 연계처리내역 : 민원상세보기(원본)\n * 추천기관 : 경찰청\n * 이송(추천...,NaN,NaN,NaN,...,0,0,0,0,0,False,0,광주광역시_동구,#동구문화센터 건너편에 신호등을 보강해주시면 감사하겠습니다.\n안녕하세요 관리자님....,398,434,260,"[-0.0103302001953125, -0.0032367706298828125, ...","[0.0015172958374023438, 0.00466156005859375, -...","[-0.01221466064453125, 0.0003306865692138672, ..."
1,1,84290,소태동 인도 정비요청(19년도에 요청~해야한다고 답변~21년 현재도 안함),김양희,2021-03-26,답변완료,안녕하세요~ 소태동 아이조움 주민입니다. 19년도 12월쯤??에 인도가 파손되어 넘...,안녕 하 `지역` 아이 주민 에 인도 파손 되 넘어지 경험 있 인도 파손 부분 인도...,안녕 하시 어요 ~ `지역` 아이 조ㅁ주민 이ㅂ니다 . 19년도 12월 쯤? ? ...,True,1. 평소 구정에 협조하여 주심에 감사드립니다.2. 귀하께서 요청하신 학동 1-1 ...,도시관리국 건설과,2021-03-30 18:46,2.020000e+17,최형호,...,0,0,0,0,0,False,0,광주광역시_동구,#소태동 인도 정비요청(19년도에 요청~해야한다고 답변~21년 현재도 안함)\n안녕...,397,439,230,"[-0.00804901123046875, -0.020599365234375, -0....","[-0.00418853759765625, -0.03228759765625, -0.0...","[-0.005916595458984375, -0.040130615234375, 0...."


In [26]:
data_able.to_csv(f"{path}{file}_embed.csv")
data_able.to_json(f"{path}{file}_embed.json")